<h1><center>Vietnam Stock Market - Momentum Stategy</center></h1>
<h3><i>Content:</i></h3>
<h3>1. Problem statement</h3>
  <li>What is momentum</li>
  <li>Why & How you find it</li>
<h3>2. Data wrangling & EDA</h3>
  <li>Candidate selection</li>
  <li>Historgram</li>
  <li>View time series/cross sectional for pattern recognition</li>
  <li>Bolinger band, high-low band (breakout)</li>
  <li>Hypothesis refinement</li>
<h3>3. Stategy design</h3>
  <p>Risk model</p>
  <p>Alpha model</p>
  <p>Multi factor model</p>
  <p>Value at risk (volatility sizing)</p>
  <ul><li>Key metrics</li>
    <ol><li>CARG</li>
    <li>Standard Deviation</li>
    <li>Downside Deviation</li>
    <li>Sharpe Ratio</li>
    <li>Sortino Ratio (MAR = 5%)</li>
    <li>Worst Drawdown</li>
    <li>Best Month Return</li>
    <li>Profitable Months</li></ol></ul>
<h3>4. Backtesting</h3>
<h3>5. Conclusion</h3>

<h2>1. Problem statement</h2>  
<b>Investors will continue to suffer behavioral bias</b>
<p>Vietnam stock exchange is one of the most potential raising up players among frontier markets. The market is in the developing stage and lacking massive & advanced financial tools such as algo-trading (quantitative trading), an alternative of fundamental & technical analysis. Quant-trading with three main alpha factors: Value, Size, Momentum aiming an initial glance for investors on how the massive trading algorithm from the mature market is able to apply and beating the benchmark in frontier one.</p>

<b>What is momentum</b>  
<b>Why is momentum</b>  
<b>How to find momentum patterns</b>  

<p>To accomplish these tasks, the following steps need to be accomplished: Hypothesis construction; Data wrangling & EDA, Strategy design; Backtesting .</p>
<p>The data was collected from the <a href=https://www.vndirect.com.vn/portal/thong-ke-thi-truong-chung-khoan/lich-su-gia.shtml>vndirect</a>
& <a href=https://s.cafef.vn/Lich-su-giao-dich-VNINDEX-1.chn#data>cafef</a</p>

<h2>2.  Data wrangling & EDA</h2>  

In [1]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

import datetime

### Load Data
The data we use for most of the projects is end of day data. This contains data for 621 stocks, but we'll be looking at stocks in the VN100. We also made things a little easier to run by narrowing down our range of time period instead of using all of the data.

In [3]:
engine = sqlalchemy.create_engine('sqlite:///' + 'vn621')
# pd.read_sql('FPT', engine).head(2)
# pd.read_sql('select * from FPT', engine)

,date,open,high,low,close,volume,unadjusted,putthrough,ticker,year,quarter,dividend
0,2006-12-13,19.225752,19.225752,19.225752,19.225752,83530,400.0,0,FPT,2006,4,NaN
1,2006-12-14,20.187039,20.187039,20.187039,20.187039,280710,420.0,0,FPT,2006,4,NaN


In [27]:
tickers = np.array(pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'",engine)).flatten()

<h2>3. Stategy design</h2>
<h3>3.1. Key metrics</h3>

<h5>CARG</h5>

<h2>4. Backtesting</h2>

<h2>5. Conclusion</h2>